# Track an experiment while training a Pytorch model with a SageMaker Training Job

This notebook shows how you can use the SageMaker SDK to track a Machine Learning experiment using a Pytorch model trained in a SageMaker Training Job with Script mode, where you will provide the model script file.

We introduce two concepts in this notebook -

* *Experiment:* An experiment is a collection of runs. When you initialize a run in your training loop, you include the name of the experiment that the run belongs to. Experiment names must be unique within your AWS account. 
* *Run:* A run consists of all the inputs, parameters, configurations, and results for one iteration of model training. Initialize an experiment run for tracking a training job with Run(). 


To execute this notebook in SageMaker Studio, you should select the `PyTorch 1.12 Python 3.8 CPU Optimizer` image.


You can track artifacts for experiments, including datasets, algorithms, hyperparameters and metrics. Experiments executed on SageMaker such as SageMaker training jobs are automatically tracked and any existen SageMaker experiment on your AWS account is automatically migrated to the new UI version.

In this notebook we will demonstrate the capabilities through an MNIST handwritten digits classification example. The notebook is organized as follow:

1. Train a Convolutional Neural Network (CNN) Model and log the model training metrics
1. Tune the hyperparameters that configures the number of hidden channels and the optimized in the model. Track teh parameter's configuration, resulting model loss and accuracy and automatically plot a confusion matrix using the Experiments capabilities of the SageMaker SDK.
1. Analyse your model results and plot graphs comparing your model different runs generated from the tunning step 3.

## Runtime
This notebook takes approximately 20 minutes to run.

## Contents
1. [Install modules](#Install-modules)
1. [Setup](#Setup)
1. [Create model training script](#Create-model-training-script)
1. [Train model with Run context](#Train-model-with-Run-context)
1. [Contact](#Contact)

## Install modules

Let's ensure we have the latest SageMaker SDK available, including the SageMaker Experiments functionality

In [ ]:
import sys

# update boto3 and sagemaker to ensure latest SDK version
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install --upgrade boto3
!{sys.executable} -m pip install --upgrade sagemaker
!{sys.executable} -m pip install torch
!{sys.executable} -m pip install torchvision

## Setup

Import required libraries and set logging and experiment configuration

SageMaker Experiments now provides the `Run` class that allows you to create a new experiment run. 

In [ ]:
from sagemaker.pytorch import PyTorch
from sagemaker.experiments.run import Run, load_run
from sagemaker.session import Session
from sagemaker import get_execution_role
from sagemaker.utils import unique_name_from_base

role = get_execution_role()
region = Session().boto_session.region_name


# set new experiment configuration
experiment_name = unique_name_from_base("training-job-experiment")
run_name = "run-example"
print(experiment_name)

## Check model training script
* Optional Step: check *`mnist.py`* using the cell below, the pytorch script file to train our model.

In [ ]:
%load  ./script/mnist.py

The cell above saves the `mnist.py` file to our script folder. The file implements the code necessary to train our PyTorch model in SageMaker, using the SageMaker PyTorch image. It uses the `load_run` function to automatically detect the experiment configuration and `run.log_parameter`, `run.log_parameters`, `run.log_file`, `run.log_metric` and `run.log_confusion_matrix` to track the model training

## Train model with Run context

Let's now train the model with passing the experiement run context to the training job

For detailed explanation of API run, refer to source code [here](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/experiments/run.py)

In [ ]:
%%time
# Start training job with experiment setting
with Run(experiment_name=experiment_name, run_name=run_name, sagemaker_session=Session()) as run:
    estmator = PyTorch(
        entry_point="./script/mnist.py",
        role=role,
        model_dir=False,
        framework_version="1.12",
        py_version="py38",
        instance_type="ml.c5.xlarge",
        instance_count=1,
        hyperparameters={"epochs": 8, "hidden_channels": 5, "optimizer": "adam", "region": region},
        keep_alive_period_in_seconds=3600,
    )

    estmator.fit()

Checking the SageMaker Experiments UI, you can observe the Experiment run, populated with the metrics and parameters logged. We can also see the automatically generated outputs for the model data


<img src="images/sm_training_exp_overview.png" width="100%" style="float: left;" />
<img src="images/sm_training_inputs.png" width="100%" style="float: left;" />
<img src="images/sm_training_parameters.png" width="100%" style="float: left;" />
<img src="images/sm_training_metrics.png" width="100%" style="float: left;" />
<img src="images/sm_training_outputs.png" width="100%" style="float: left;" />

## Run multiple experiments

You can now create multiple runs of your experiment by varying a few parameters. Feel free to play with the parameters. 

In [ ]:

%%time
# Start training job with experiment setting
num=1
hidden_channels=[5,10]
optimizer=["adam", "sgd"]

for h in hidden_channels:
    for j in optimizer:
        num=num+1
        run_name_n=run_name+'-'+str(num)
        with Run(experiment_name=experiment_name, run_name=run_name_n) as run:
            print("hidden_channels-", h, " optimizer-", j)
            estmator = PyTorch(
                entry_point="./script/mnist.py",
                role=role,
                model_dir=False,
                framework_version="1.12",
                py_version="py38",
                instance_type="ml.c5.xlarge",
                instance_count=1,
                hyperparameters={"epochs": 10, "hidden_channels": h, "optimizer": j, "region": region},
                keep_alive_period_in_seconds=3600,
            )

            estmator.fit(wait=False)


## Compare the performance through Experiment UI

In the SageMaker Experiments UI, you can compare the different runs and analyze the metrics for those runs 


<img src="images/compare_experiments.png" width="100%"/>


## Bonus Point: customized analysis 

Besides all the built-in analysis withn Experiments, you can also customize your analysis and plot based on available metrics and parameter!

Below, we have provided you an example for your reference. 

This analsis is built on top of [Experiment Analysis API](https://sagemaker.readthedocs.io/en/stable/api/training/analytics.html)

In [ ]:
from script.experiment_plot import analyze_experiment

In [ ]:
analyze_experiment(experiment_name)